In [7]:
%run "../00_project_config.ipynb"
%run "05_annotations.ipynb"
%run "10_data_objects.ipynb"
%run "20_process_sample_table.ipynb"
%run "30_process_data_sources.ipynb"

MANUALLY_CURATED_FOLDERS = {
    "20230501_HGSVC_UL_ONT-UW": "UW_WH",
    "20211013_ONT_Rebasecalled": "JAX_PA",
    "20230706_HGSVC_EEE_UL_ONT": "UW_WH",
    "20230703_HGSVC_EEE_HIFI": "UW_WH",
    "20220831_JAX_HiFi": "JAX_PA"
}

AUTO_TABLE_ALL_KNOWN = PROJECT_BASE.joinpath("annotations", "autogen", "annotated_files.tsv")
AUTO_TABLE_ALL_KNOWN.parent.mkdir(parents=True, exist_ok=True)
AUTO_TABLE_ALL_FILES = PROJECT_BASE.joinpath("annotations", "autogen", "existing_files.tsv")
AUTO_TABLE_ALL_FILES.parent.mkdir(parents=True, exist_ok=True)
AUTO_TABLE_ERR_FILES = PROJECT_BASE.joinpath("annotations", "autogen", "error_files.tsv")
AUTO_TABLE_ERR_FILES.parent.mkdir(parents=True, exist_ok=True)

matched_records = []
matched_files = []
for row in KNOWN_FILES.itertuples(index=True):
    
    files_for_sample = FILES_EXIST_PER_SAMPLE[row.sample]
    matched_files = []
    num_matches = 0
    sample_batch = SAMPLES[row.sample].batch_num
    for file in files_for_sample:
        if row.read_type != file.read_type:
            continue
        if row.cell in file.file_name:
            assert row.read_type == file.read_type
            matched_records.append(
                (row.Index, sample_batch, file.data_rel_path, file.file_date, file.file_group, file.group_date, 1)
            )
            matched_files.append(file)
            num_matches += 1
    if num_matches > 1:
        for mf in matched_files:
            print(mf)
        raise ValueError(f"Multi-match: {row}")
        
    if num_matches == 1:
        matched_files[0].set_matched_entry(row.Index)
        matched_files[0].set_curator("metadata_table")
    
matched_records = pd.DataFrame.from_records(
    matched_records,
    columns=["index", "processing_batch", "file_rel_path", "file_date", "file_group", "group_date", "matched"]
)
matched_records.index = matched_records["index"]
matched_records.drop("index", inplace=True, axis=1)

MERGED = KNOWN_FILES.merge(matched_records, how="outer", left_index=True, right_index=True)
MERGED["matched"].fillna(0, inplace=True)
MERGED["processing_batch"].fillna(-1, inplace=True)
MERGED.fillna("n/a", inplace=True)
MERGED["matched"] = MERGED["matched"].astype(int)
MERGED["processing_batch"] = MERGED["processing_batch"].astype(int)

all_files_table = []
for sample, files_for_sample in FILES_EXIST_PER_SAMPLE.items():
    for sample_file in files_for_sample:
        if sample_file.matched_entry is None:
            for folder, curator in MANUALLY_CURATED_FOLDERS.items():
                if folder in str(sample_file.data_rel_path):
                    sample_file.set_matched_entry(-1)
                    sample_file.set_curator(curator)
                    break
            if sample_file.matched_entry is None:
                sample_file.set_curator("error")
        all_files_table.append(sample_file.get_table_row())
all_files_header = sample_file.get_table_header()

with open(AUTO_TABLE_ALL_KNOWN, "w") as dump_table:
    _ = dump_table.write(f"# {TODAY}\n")
    MERGED.to_csv(
        dump_table,
        sep="\t",
        header=True,
        index=True,
        index_label="index"
    )
    
ALL_FILES = pd.DataFrame.from_records(
    all_files_table,
    columns=all_files_header
)
ALL_FILES.sort_values(["sample", "read_type", "file_group", "file_name"], inplace=True)
with open(AUTO_TABLE_ALL_FILES, "w") as dump_table:
    _ = dump_table.write(f"# {TODAY}\n")
    ALL_FILES.to_csv(
        dump_table,
        sep="\t",
        header=True,
        index=False
    )
    
ERR_FILES = ALL_FILES.loc[ALL_FILES["curated_by"] == "error", :].copy()
with open(AUTO_TABLE_ERR_FILES, "w") as dump_table:
    _ = dump_table.write(f"# {TODAY}\n")
    ERR_FILES.to_csv(
        dump_table,
        sep="\t",
        header=True,
        index=False
    )


Skipping over 19320 - not enough data

Skipping over 19320 - not enough data
